In [53]:
# all important imports
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup 
from selenium import webdriver
import random
import time

In [54]:
# opens web browser
dr = webdriver.Chrome()

In [55]:
# gets some pick up lines so you dont have to bother
pick_up_phrases = 'http://pun.me/pages/cheesy-pick-up-lines.php'
page = uReq(pick_up_phrases)
bs = BeautifulSoup(page.read(),"html.parser")

pick_up_lines = list()

lines = bs.find_all('li', href = False)
for line in lines:
    if '<a' in str(line):
        continue
    else:
        pick_up_lines.append(line.text)

In [56]:
pick_up_phrases = 'https://thoughtcatalog.com/kendra-syrdal/2016/05/101-online-dating-openers-that-are-way-better-than-just-saying-hi/'
page = uReq(pick_up_phrases)
bs = BeautifulSoup(page.read(),"html.parser")

items = bs.find('div',{'class':'entry'})

lines = items.find_all('p')

for line in lines:
    pick_up_lines.append(line.text)

In [57]:
# opens the browser and lets you log in unless you login dont start next process
main_link = 'https://www.okcupid.com'
# to enter your credentials
my_url_Page = main_link+'/doubletake'
dr.get(my_url_Page)

In [58]:
people_matched = list()

In [ ]:
# check the person. If match rate is more than 80% then sends of random pick up line
for a in range (1,100):
    # person, pick up line
    
    person = dict()
    
    # to make less predictable for the webpage
    rand_timeout = random.randint(5,10)
    
    my_url_Page = main_link+'/doubletake'
    dr.get(my_url_Page)
    bs = BeautifulSoup(dr.page_source,"html.parser")
    
    match_prc = bs.find('span',{'class':'cardsummary-match-pct'}).text
    match_number = int(match_prc.replace('%',''))
    
    # if the match rate is more than 85
    if match_number > 85:
        # just in case something goes wrong or response is not delivered in time
        try:
            match_profile = bs.find('div',{'class':'cardsummary-item cardsummary-profile-link'})
            new_link = main_link + match_profile.find('a',href = True)['href']
            dr.get(new_link)
            button_like = dr.find_element_by_id('like-button')
            button_like.click()

            time.sleep(rand_timeout)

            # to make it more customizable we can add name in from.
            bsn = BeautifulSoup(dr.page_source,"html.parser")
            name = bsn.find('div',{'class':'userinfo2015-basics-username'}).text.strip()
            pick_up_line = random.choice(pick_up_lines).strip()
            
            person['name'] = name
            person['pick_up_line'] = pick_up_line   
            people_matched.append(person)
            
            time.sleep(rand_timeout)

            text = dr.find_element_by_class_name('messenger-composer')
            text.send_keys("Hey ",name,'. ',pick_up_line)
            time.sleep(1)
            button_send = dr.find_element_by_class_name('messenger-toolbar-send')
            button_send.click()
            
            time.sleep(rand_timeout)
            
        except:
            pass
    else:
        try:
            button_reject = dr.find_element_by_class_name('cardactions-action')
            button_reject.click()

            time.sleep(rand_timeout)
        except:
            pass

In [65]:
print(people_matched)

[{'name': 'Rubi', 'pick_up_line': "My love for you is like diarrhoea, I just can't hold it in."}, {'name': 'Sonja', 'pick_up_line': 'Fuck, marry, kill. Ellen, Oprah, Chelsea Handler?'}, {'name': 'Lena', 'pick_up_line': 'Are you craving Pizza? Because I’d love to get a pizz-a you'}, {'name': 'Rieke', 'pick_up_line': "Did you hear of the new disease called beautiful, I think you're infected."}, {'name': 'El', 'pick_up_line': 'Swear to drunk\xa0I’m not God…just intoxicated by you.'}, {'name': 'Mila', 'pick_up_line': 'You know what would be great? Talking to you.'}, {'name': 'Melina', 'pick_up_line': 'Remember me? Oh, that’s right, I’ve met you only in my dreams.'}, {'name': 'Yhenni', 'pick_up_line': 'What’s something no one would guess about you from a first impression?'}, {'name': 'Amy', 'pick_up_line': 'Want to engage in a textually active relationship?'}, {'name': 'Amy', 'pick_up_line': 'Sorry for all of the gross messages you probably get here 😕'}, {'name': 'Sophie', 'pick_up_line': '